In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# data.py

In [28]:
import tensorflow as tf
from tensorflow.keras import datasets

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 1000

def normalization(images, labels):
    images = images / 255
    return images, labels

def load_mnist(flatten=True) -> (tf.data.Dataset, tf.data.Dataset):
    """ return train_ds, test_ds """
    (train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
    
    if flatten:
        train_images = train_images.reshape((60000, 28 * 28))
    else:
        train_images = train_images.reshape((60000, 28, 28, 1))
    
    train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
    
    train_ds = train_ds.map(normalization).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    test_ds = test_ds.map(normalization).batch(BATCH_SIZE)
    
    return train_ds, test_ds

In [29]:
train_ds, test_ds = load_mnist(flatten=True)

In [30]:
next(iter(train_ds.take(1)))

(<tf.Tensor: shape=(64, 784), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(64,), dtype=uint8, numpy=
 array([0, 1, 2, 9, 2, 1, 6, 0, 4, 2, 5, 0, 0, 1, 7, 7, 0, 4, 5, 4, 3, 0,
        3, 8, 4, 0, 9, 2, 9, 9, 7, 9, 4, 3, 6, 7, 8, 7, 5, 4, 8, 0, 5, 6,
        9, 3, 4, 1, 6, 5, 7, 5, 6, 1, 0, 6, 1, 8, 1, 9, 9, 0, 8, 3],
       dtype=uint8)>)

# AE.py

In [6]:
import os

print(os.path.abspath("."))
cur_dir = os.path.abspath(".")
name = "AE-stacked"
print(os.path.join(cur_dir, f"weights/{name}.tf"))


/tf/tensorflow/models/1.AE
/tf/tensorflow/models/1.AE/weights/AE-stacked


In [128]:
import os
import tensorflow as tf

cur_dir = os.path.abspath(".")

class AE(tf.keras.Model):
    def __init__(self, name):
        super(AE, self).__init__(name=name)
        self.cur_dir = os.path.abspath(".")
        self.filepath = os.path.join(cur_dir, "weights")
        
    def save_weight(self, epoch, loss):
        filepath = os.path.join(self.filepath, f"epochs:{epoch}_loss:{loss}")
        try:
            self.save_weights(filepath, overwrite=True,save_format="tf")
        except ImportError as e:
            print("h5py is not available and the weight file is in HDF5 format.", e, sep='\n')
    
    def load_weight(self):
        self.load_weights(self.filepath, by_name=False)

# AE-undercomplete.py

In [ ]:
n_inputs = 784
n_hidden = 100
n_outputs = n_inputs

class AE_UC(AE):
    def __init__(self):
        super().__init__("AE-undercomplete")
        
        self.hidden = tf.keras.layers.Dense(n_hidden)
        self.outputs = tf.keras.layers.Dense(n_outputs)
        
    def call(self, x):
        x = self.hidden(x)
        return self.outputs(x)

# train.py

In [31]:
ae_uc(next(iter(train_ds.take(1)))[0])

<tf.Tensor: shape=(64, 784), dtype=float32, numpy=
array([[-0.30250287,  0.14121279, -0.17230012, ..., -0.02988997,
         0.395446  , -0.11788203],
       [-0.24505225, -0.04016065, -0.1062265 , ..., -0.45540535,
         0.21555413, -0.23108496],
       [-0.34005308, -0.36352518,  0.06748123, ...,  0.05447821,
         0.23479328, -0.05959939],
       ...,
       [-0.32307187, -0.09876315, -0.19627583, ..., -0.2068078 ,
         0.20434165, -0.26404977],
       [-0.01913455, -0.075371  ,  0.00666739, ..., -0.17684942,
        -0.06389791, -0.23046623],
       [-0.07081459, -0.06656758,  0.01953682, ..., -0.12992321,
        -0.14620791, -0.33002508]], dtype=float32)>

In [ ]:
for model_class in AE.__subclasses__():
    model = model_class()
    train(model)
    model.save_weight()

In [ ]:
def train():
    

In [114]:
loss_object = tf.keras.losses.MSE

def loss(model, x, y):
    y_ = model(x)
    return loss_object(y_true=y, y_pred=y_)

for x, y in train_ds.take(1):
    l = loss(ae_uc, x, x)
    print(l)

tf.Tensor(
[0.25073615 0.12835439 0.18821526 0.24678671 0.22956376 0.11011429
 0.44523752 0.08580049 0.3054949  0.29237276 0.2982693  0.09136851
 0.45562643 0.30502653 0.18890291 0.37130588 0.26691556 0.17026897
 0.17850773 0.1893518  0.2399322  0.22588003 0.21046658 0.24306823
 0.37563515 0.19968452 0.15425327 0.12994182 0.08081746 0.22711037
 0.09924209 0.19295368 0.23766322 0.2072405  0.09583682 0.2974602
 0.16440491 0.32772648 0.08594456 0.12127063 0.20471005 0.16727671
 0.18518576 0.14465654 0.06471962 0.24219517 0.13521004 0.30665264
 0.16529243 0.30386615 0.52407146 0.3094855  0.27067193 0.2752135
 0.17686212 0.11787312 0.13846685 0.23650107 0.07569847 0.39356014
 0.07240282 0.2558876  0.04085666 0.38552153], shape=(64,), dtype=float32)


In [115]:
def grad(model, x, y):
    with tf.GradientTape() as tape:
        loss_value = loss(model, x, y)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [116]:
loss_value, grads = grad(ae_uc, x, x)

print("단계: {}, 초기 손실: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, ae_uc.trainable_variables))

print("단계: {},      손실: {}".format(optimizer.iterations.numpy(),
                                          loss(ae_uc, x, x).numpy()))

단계: 0, 초기 손실: [0.25073615 0.12835439 0.18821526 0.24678671 0.22956376 0.11011429
 0.44523752 0.08580049 0.3054949  0.29237276 0.2982693  0.09136851
 0.45562643 0.30502653 0.18890291 0.37130588 0.26691556 0.17026897
 0.17850773 0.1893518  0.2399322  0.22588003 0.21046658 0.24306823
 0.37563515 0.19968452 0.15425327 0.12994182 0.08081746 0.22711037
 0.09924209 0.19295368 0.23766322 0.2072405  0.09583682 0.2974602
 0.16440491 0.32772648 0.08594456 0.12127063 0.20471005 0.16727671
 0.18518576 0.14465654 0.06471962 0.24219517 0.13521004 0.30665264
 0.16529243 0.30386615 0.52407146 0.3094855  0.27067193 0.2752135
 0.17686212 0.11787312 0.13846685 0.23650107 0.07569847 0.39356014
 0.07240282 0.2558876  0.04085666 0.38552153]
단계: 1,      손실: [2.6196725  0.9489364  1.3011646  2.5595345  1.8022598  0.91660386
 5.1551476  0.5357138  4.5109982  3.3654714  2.8366475  0.6796931
 5.95916    2.9916115  2.2253213  4.7588344  2.5718105  1.8523843
 2.1750662  2.446693   2.246922   2.756946   2.5022461  3

In [125]:
ae_uc.trainable_variables[0].device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [120]:
ae_uc.summary()

Model: "AE-undercomplete"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              multiple                  78500     
_________________________________________________________________
dense_9 (Dense)              multiple                  79184     
Total params: 157,684
Trainable params: 157,684
Non-trainable params: 0
_________________________________________________________________


In [127]:
train_loss_results = []

ae_uc = AE_UC()

epochs = 30

for epoch in range(epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    
    for x, _ in train_ds:
        loss_value, grads = grad(ae_uc, x, x)
        optimizer.apply_gradients(zip(grads, ae_uc.trainable_variables))
        
        epoch_loss_avg(loss_value)
        
    train_loss_results.append(epoch_loss_avg.result())
    
    print("에포크 {:03d}: 손실: {:.3f}".format(epoch, epoch_loss_avg.result()))

에포크 000: 손실: 0.803
에포크 001: 손실: 0.019
에포크 002: 손실: 0.013
에포크 003: 손실: 0.010
에포크 004: 손실: 0.008
에포크 005: 손실: 0.007
에포크 006: 손실: 0.007
에포크 007: 손실: 0.006
에포크 008: 손실: 0.006
에포크 009: 손실: 0.007
에포크 010: 손실: 0.007
에포크 011: 손실: 0.008
에포크 012: 손실: 0.009
에포크 013: 손실: 0.152
에포크 014: 손실: 0.009
에포크 015: 손실: 0.007
에포크 016: 손실: 0.006
에포크 017: 손실: 0.006
에포크 018: 손실: 0.006
에포크 019: 손실: 0.006
에포크 020: 손실: 0.006
에포크 021: 손실: 0.007
에포크 022: 손실: 0.007
에포크 023: 손실: 0.009
에포크 024: 손실: 0.312
에포크 025: 손실: 0.007
에포크 026: 손실: 0.007
에포크 027: 손실: 0.007
에포크 028: 손실: 0.006
에포크 029: 손실: 0.006


In [132]:
ae_uc.save_weights("./weights/something.weights")